In [5]:
# import the libraries

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np


Create Model

In [2]:
# old model
model = keras.Sequential([
    keras.layers.Rescaling(1./255),
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, (3, 3), activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid"),


])

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop", metrics=["accuracy"])


2023-03-13 14:25:14.974461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 14:25:15.120001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 14:25:15.120421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 14:25:15.122485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 14:25:15.122871: I tensorflow/compiler/xla/stream_executo

DATA LOADING

In [3]:
# Load the data
definitions = pd.read_csv('OPTED-Dictionary.csv')
simpDef1 = pd.read_excel('ChildFriendlyDefinitions.xlsx', sheet_name='Sheet1')
simpDef2 = pd.read_json('data.json')


DATA PREPROCESSING (Everything below this only needs to be run once. A CSV will be created with the full dataset)

In [4]:
import pandas as pd


def cleanDataframe(df):
    df_copy = df

    # may need to take the '-' out
    regex = "\[(.*?)\]|[0-9!@#$%^&*?\/=+\-]|\((.*?)\)|\{(.*?)\}|\<(.*?)\>"

    df_copy = df_copy.replace(
        to_replace=regex, value="", regex=True).dropna()  # remove illegal chars

    df_copy.word = df_copy.word.str.lower()  # lower case everything

    df_copy = df_copy.sort_values('word', ascending=True)
    df_copy = df_copy.drop_duplicates(subset='word', keep='first')

    return df_copy.reset_index().drop(['index'], axis=1)



ACTUAL DEFINITIONS

In [5]:
definitions_filter = definitions.drop(['Count', 'POS'], axis=1)
definitions_filter['word'] = definitions_filter['Word']
definitions_filter['definition'] = definitions_filter['Definition']

definitions_filter = definitions_filter.drop(['Word', 'Definition'], axis=1)
definitions_filter = cleanDataframe(df=definitions_filter)

regexQuote = "^\"|\"$"
definitions_filter = definitions_filter.replace(
    to_replace=regexQuote, value="", regex=True)
definitions_filter


,word,definition
0,'em,An obsolete or colloquial contraction of the o...
1,'gainst,A contraction of Against.
2,'mongst,See Amongst.
3,'neath,An abbreviation of Beneath.
4,'s,A contraction for is or for has.
...,...,...
111468,zymotic,Of pertaining to or caused by fermentation.
111469,zyophyte,Any plant of a proposed class or grand divisio...
111470,zythem,See Zythum.
111471,zythepsary,A brewery.


SIMPLIFIED DEFINITIONS

Part A

In [6]:
# Filter simpDef1
simpDef1_Filter = simpDef1.drop(['Exemplar'], axis=1)
simpDef1_Filter['word'] = simpDef1_Filter['Word']
simpDef1_Filter['definition'] = simpDef1_Filter['Child Friendly Definition']
simpDef1_Filter = simpDef1_Filter.drop(
    ['Word', 'Child Friendly Definition'], axis=1)
simpDef1_Filter = cleanDataframe(simpDef1_Filter)
simpDef1_Filter


,word,definition
0,accessible,When something is accessible it means anyone c...
1,accommodate,You accommodate when you change something that...
2,accomplish,"If you accomplish something, you succeed in do..."
3,achieve,"If you achieve something, you succeed in doing..."
4,acre,An acre is a very large area of land about the...
...,...,...
162,value,The value of a place or thing is how much mone...
163,verify,"If you verify something, you make sure that it..."
164,vigilant,Someone who is vigilant pays careful attention...
165,visible,"When something is visible, you can see it."


Part B

In [7]:
simpDef2_filter = simpDef2
simpDef2_Filter = cleanDataframe(simpDef2_filter)
simpDef2_Filter


,word,definition
0,'s,a suffix used to form the possessive of most s...
1,'tis,"shortened form of ""it is."""
2,'twas,"shortened form of ""it was."""
3,a,the first letter of the English alphabet.
4,a dime a dozen,plentiful and easy to get; common; cheap.
...,...,...
13907,zone,an area that is divided from other areas becau...
13908,zoo,"a place where living animals, especially wild ..."
13909,zoology,the science and study of animals.
13910,zoom,to move quickly while making a low humming sou...


Getting Dataset ready for training

In [8]:
# Basically we are combining all the dataset together and putting it into a csv
# Doing this because i don't want to constantly run this script (takes up RAM that I dont have)

df = pd.DataFrame(columns=['word', 'definition', 'simplified_definition'])

for index, row in simpDef1_Filter.iterrows():
    defs = definitions_filter[row['word'] ==
                              definitions_filter.word]['definition']
    if defs.count() >= 1:
        w = row['word']
        simpDefs = row['definition']
        defs = defs.values[0]
        df = pd.concat([df, pd.DataFrame([[w, defs, simpDefs]], columns=[
                       'word', 'definition', 'simplified_definition'])], ignore_index=True)

for index, row in simpDef2_Filter.iterrows():
    defs = definitions_filter[row['word'] ==
                              definitions_filter.word]['definition']
    if defs.count() >= 1:
        w = row['word']
        simpDefs = row['definition']
        defs = defs.values[0]
        df = pd.concat([df, pd.DataFrame([[w, defs, simpDefs]], columns=[
                       'word', 'definition', 'simplified_definition'])], ignore_index=True)

df


KeyboardInterrupt: 

In [3]:
df.to_csv("fullDataset.csv", index=False)


NameError: name 'df' is not defined

Training the Algorithm

In [7]:
# load csv
fullDs = pd.read_csv("fullDataset.csv")


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf


def convertTextToNumbers(words, tokenizer=None, padding=None, isTrainY=False):
    # Create a tokenizer and fit it on the entire text corpus
    if tokenizer == None:
        #tokenizer = Tokenizer(split=' ')
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(words)

    # Convert the text to sequences of integers
    sequences = tokenizer.texts_to_sequences(words)

    if isTrainY:
        tokenizer.word_index['<start>'] = len(
            tokenizer.word_index) + 1
        tokenizer.word_index['<end>'] = len(
            tokenizer.word_index) + 1
        sequences = [[tokenizer.word_index['<start>']] +
                     seq + [tokenizer.word_index['<end>']] for seq in sequences]

    # Get the word index mapping
    word_index = tokenizer.word_index

    # Making sure all inputs are of the same length
    if padding == None:
        sequences = tf.keras.preprocessing.sequence.pad_sequences(
            sequences, padding='post')
    else:
        sequences = tf.keras.preprocessing.sequence.pad_sequences(
            sequences, maxlen=padding, padding='post')
    return (sequences, word_index, tokenizer)


In [5]:
# Input data
input_texts = ['dog is an animal', 'cat is a mammal', 'fish lives in water']
simplified_texts = ['dog is animal', 'cat is mammal', 'fish lives in water']

# Tokenize input and output data
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)
max_len_input = max(len(seq) for seq in input_seq)
input_pad_seq = pad_sequences(input_seq, maxlen=max_len_input, padding='post')

simp_eng_tokenizer = Tokenizer()
simp_eng_tokenizer.fit_on_texts(simplified_texts)
simp_eng_seq = simp_eng_tokenizer.texts_to_sequences(simplified_texts)
max_len_output = max(len(seq) for seq in simp_eng_seq)
simp_eng_pad_seq = pad_sequences(
    simp_eng_seq, maxlen=max_len_output, padding='post')

# Add start and end tokens to decoder input and target data
decoder_input_data = np.zeros(
    (len(simplified_texts), max_len_output), dtype='int32')
decoder_target_data = np.zeros((len(simplified_texts), max_len_output, len(
    simp_eng_tokenizer.word_index) + 1), dtype='float32')


for i, text in enumerate(simplified_texts):
    tokens = ['<start>'] + text.split() + ['<end>']
    for j, token in enumerate(tokens):
        decoder_input_data[i, j] = simp_eng_tokenizer.word_index.get(token, 0)
        if j > 0:
            k = simp_eng_tokenizer.word_index.get(token, 0)
            decoder_target_data[i, j - 1, k] = 1.0

# Print example input and target data for the first sample
print('Encoder input data:', input_pad_seq[0])
print('Decoder input data:', decoder_input_data[0])
print('Decoder target data:')
for j in range(max_len_output):
    print(simp_eng_tokenizer.index_word[np.argmax(
        decoder_target_data[0, j])], end=' ')


NameError: name 'pad_sequences' is not defined

In [9]:
# split training set (do not need test) - we will provide a definition and evaluate it ourselves
# we also may not need a validation set since i dont know if it will evaluate it properly.
fullDs = fullDs.sample(frac=1)  # shuffle the dataset
fullDs = fullDs.astype(str)

# the two arrays below are tokenized and padded for the algorithm
full_train_x, defWordIndex, defTokenizer = convertTextToNumbers(
    fullDs['definition'].values[0:10000])

full_train_y, simpDefWordIndex, simpDefTokenizer = convertTextToNumbers(
    fullDs['simplified_definition'].values[0:10000], isTrainY=True)

full_train_y_RAW = fullDs['simplified_definition'].values[0:10000]
#decoder_target_data_y = createOneHotEncodingVector(full_train_y, len(simpDefWordIndex), simpDefWordIndex)
#full_train_x = full_train_x.astype('int')
#full_train_y = full_train_y.astype('int')


In [35]:
len(fullDs)


10918

In [10]:
full_train_x[0]


array([6350, 2709, 3866,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [8]:
full_train_y[0]


array([8993,  733,   44,    3, 8994,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [34]:
train_ratio = 0.8

# Split the DataFrame into training and validation sets
train_x = full_train_x[:int(len(full_train_x) * train_ratio)]
train_y = full_train_y[:int(len(full_train_y) * train_ratio)]
val_x = full_train_x[int(len(full_train_x) * train_ratio):]
val_y = full_train_y[int(len(full_train_y) * train_ratio):]

# Convert the Pandas DataFrame to TensorFlow Dataset
#train_ds = tf.data.Dataset.from_tensor_slices((train_df.values[:, :-1], train_df.values[:, -1]))
#val_ds = tf.data.Dataset.from_tensor_slices((val_df.values[:, :-1], val_df.values[:, -1]))
print(len(fullDs['definition']))  # 10918
print(len(train_x))  # 8734
print(len(train_y))  # 8734
print(len(val_x))  # 2184
print(len(val_y))  # 2184


10918
8000
8000
2000
2000


In [11]:
print(full_train_x[0])  # def (should be head(1))
print(full_train_y[0])  # simp def

print(defTokenizer.sequences_to_texts([full_train_x[0]])[0])
print(simpDefTokenizer.sequences_to_texts([full_train_y[0]])[0])
print("\n\n\n")
fullDs.head(1)


[6350 2709 3866    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[8691    6  533  210    3 8692    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0

,word,definition,simplified_definition
1987,cling,Adherence; attachment; devotion.,to stick closely.


In [12]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Lambda
import tensorflow as tf
from tensorflow.keras.models import Model
from keras import backend as K


def createModel(encoding_token, decoding_token, embedding_dim, latent_dim):

    # Define input sequence
    inputs = Input(shape=(None,))

    # Define encoder embedding layer
    enc_emb = Embedding(input_dim=encoding_token,
                        output_dim=embedding_dim)(inputs)

    # Define encoder LSTM
    encoder_lstm = LSTM(latent_dim, return_state=True)
    _, state_h, state_c = encoder_lstm(enc_emb)

    # Discard encoder outputs and only keep states
    encoder_states = [state_h, state_c]

    # Define decoder input sequence
    decoder_inputs = Input(shape=(None,))

    # Define decoder embedding layer
    dec_emb_layer = Embedding(input_dim=decoding_token,
                              output_dim=embedding_dim)
    dec_emb = dec_emb_layer(decoder_inputs)

    # Define decoder LSTM
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

    # Define decoder output layer
    #flat = Flatten()(decoder_outputs)

    #decoder_dense = Dense(decoding_token, activation='softmax')

    decoder_dense = TimeDistributed(Dense(decoding_token, activation='linear'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model([inputs, decoder_inputs], decoder_outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    return model


encoding_token = len(defWordIndex)
decoding_token = len(simpDefWordIndex)
print(encoding_token)
print(decoding_token)
# usually 50 - 500 higher = more complex relation but higher chance of overfitting.
embedding_dim = 100
# usually 128 - 256 higher = more complex relation but higher chance of overfitting.
latent_dim = 170
model = createModel(encoding_token=encoding_token, decoding_token=decoding_token,
                    embedding_dim=embedding_dim, latent_dim=latent_dim)
model.summary()


17600
8692
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    1760000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    869200      ['input_2[0][0]']                
                                                                                   

In [13]:
# use this one

from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Lambda
import tensorflow as tf
from tensorflow.keras.models import Model
from keras import backend as K


def createModel(encoding_token, decoding_token, embedding_dim, latent_dim):

    # Define input sequence
    inputs = Input(shape=(None,))

    # Define encoder embedding layer
    enc_emb = Embedding(encoding_token, latent_dim, mask_zero=True)(inputs)

    # Define encoder LSTM
    encoder_lstm = LSTM(latent_dim, return_state=True)
    _, state_h, state_c = encoder_lstm(enc_emb)

    # Discard encoder outputs and only keep states
    encoder_states = [state_h, state_c]

    # Define decoder input sequence
    decoder_inputs = Input(shape=(None,))

    # Define decoder embedding layer
    dec_emb_layer = Embedding(decoding_token,
                              output_dim=embedding_dim)
    dec_emb = dec_emb_layer(decoder_inputs)

    # Define decoder LSTM
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

    # Define decoder output layer
    # flat = Flatten()(decoder_outputs)

    # decoder_dense = Dense(decoding_token, activation='softmax')

    decoder_dense = TimeDistributed(
        Dense(decoding_token, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model([inputs, decoder_inputs], decoder_outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    print(model.summary())
    return model


encoding_token = len(defWordIndex) + 1
decoding_token = len(simpDefWordIndex) + 1
print(encoding_token)
print(decoding_token)
# usually 50 - 500 higher = more complex relation but higher chance of overfitting.
embedding_dim = 50
# usually 128 - 256 higher = more complex relation but higher chance of overfitting.
latent_dim = 128
model = createModel(encoding_token=encoding_token, decoding_token=decoding_token,
                    embedding_dim=embedding_dim, latent_dim=latent_dim)


17601
8693
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 128)    2252928     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 50)     434650      ['input_4[0][0]']                
                                                                                 

In [7]:
for index, sentence in enumerate(full_train_y_RAW[0:10]):
    tokens = sentence.split(' ')
    for j, token in enumerate(tokens):
        print(token)


the
period
between
late
afternoon
and
nightfall. 
to
climb
with
difficulty
or
in
an
awkward
way,
using
hands
and
feet.
completely;
entirely.
loose,
light,
fluffy
matter
such
as
fibers
or
hairs.
not
valuable
or
important;
insignificant.
not
containing
anything;
empty. 
to
hold
or
squeeze
with
the
arms
in
a
loving
way;
embrace. 
soldiers
on
foot,
or
the
branch
of
the
military
to
which
they
belong.
being
the
place
of
birth
or
origin. 
the
position
between
second
and
third
base
in
baseball
or
softball,
or
the
player
in
this
position.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


2023-03-13 14:27:53.318053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 14:27:53.326475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 14:27:53.326716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [14]:
def data_generator(input_sequences, target_sequences, target_RAW, batch_size, simpWordIndex, lengthOfSentence):
    while True:
        for i in range(0, len(target_RAW), batch_size):

            decoder_target_data = np.zeros(
                (batch_size, lengthOfSentence, len(simpWordIndex)+1), dtype='float32')
            for index, sentence in enumerate(target_RAW[i:i+batch_size]):
                #print(sentence)
                tokens = sentence.split(' ')
                for j, token in enumerate(tokens):
                    temp = simpWordIndex.get(token, 0)
                    if temp > 0:

                        decoder_target_data[index, j, temp] = 1.0
                        
            yield [input_sequences[i:i+batch_size], target_sequences[i:i+batch_size]], decoder_target_data


In [15]:
batch_size = 50
steps_per_epoch = len(full_train_x) // batch_size

generator = data_generator(full_train_x, full_train_y, full_train_y_RAW,
                           batch_size, simpDefWordIndex, len(full_train_y[0]))
history = model.fit(generator, epochs=100, batch_size=batch_size, steps_per_epoch=steps_per_epoch)


2023-03-16 10:18:58.128344: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 173860000 exceeds 10% of free system memory.


Epoch 1/100


2023-03-16 10:19:04.932970: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 173860000 exceeds 10% of free system memory.
2023-03-16 10:19:05.189789: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 173860000 exceeds 10% of free system memory.
2023-03-16 10:19:05.314768: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2023-03-16 10:19:11.545986: I tensorflow/compiler/xla/stream_executor/cuda/cuda_d

  2/200 [..............................] - ETA: 1:25 - loss: 0.8934   

2023-03-16 10:19:31.440578: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 173860000 exceeds 10% of free system memory.


  3/200 [..............................] - ETA: 1:22 - loss: 0.8803

2023-03-16 10:19:31.821397: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 173860000 exceeds 10% of free system memory.


200/200 [==============================] - 95s 317ms/step - loss: 0.6178
Epoch 2/100
200/200 [==============================] - 45s 226ms/step - loss: 0.5455
Epoch 3/100
 14/200 [=>............................] - ETA: 44s - loss: 0.5293

KeyboardInterrupt: 

In [46]:
model.save('model.h5')

In [ ]:
def createOneHotEncodingVector(simpDef, maxLengthOutput, wordIndex):
    decoder_target_data = np.zeros(
        (len(simpDef), maxLengthOutput, len(wordIndex) + 1), dtype='float32')
    for i, word in enumerate(simpDef):
        for j, token in enumerate(word):
            index = wordIndex.get(token, 0)
            if index > 0:
                decoder_target_data[i, j-1, index] == 1.0
    return decoder_target_data


In [10]:
len(full_train_y[0])


66

In [11]:
len(full_train_y)


100

In [12]:
len(simpDefWordIndex)


602

In [33]:
full_train_y_RAW[0]


'at one time in the past; formerly.\xa0'

In [39]:
t = full_train_y_RAW[0].split(' ')[0]
print(t)
simpDefWordIndex.get(t, 0)


at


36

In [136]:
len(train_x[0])


118

Data Prediction

In [22]:
import tensorflow as tf
new_model = tf.keras.models.load_model(
    './model.h5')


In [58]:
import numpy as np
data, tokens, tokenizer = convertTextToNumbers(
    ["West African ceremonial trumpet"], defTokenizer, padding=len(full_train_x[0]))
print(data)
print(defTokenizer.sequences_to_texts(data))

inputdata = np.array(data).reshape(1, -1)

outputdata = np.zeros(shape=(1, len(full_train_y[0])))
outputdata[0, 0] = simpDefTokenizer.word_index['<start>']
#outputdata = np.reshape(outputdata, (1,1))


newSentence = model.predict([inputdata, outputdata])
print(newSentence)
#print(simpDefTokenizer.sequences_to_texts(newSentence))

# Assuming you have already preprocessed and tokenized the sentence
#test_sentence = "the cat jumped over the moon"
#test_sentence_tokens = tokenizer.texts_to_sequences([test_sentence])[0]
#test_sentence_tokens = np.array(test_sentence_tokens).reshape(1, -1)

# Initialize the decoder input with the start token
#decoder_input = np.zeros(shape=(1, max_summary_len))
#decoder_input[0, 0] = summary_tokenizer.word_index['start']

# Reshape the decoder input to have a length of 1
#decoder_input = np.reshape(decoder_input, (1, 1))


[[  629  2244 16425  2983     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0]]
['west african ceremonial trumpet']
1/1 [==============================] - 0s 35ms/step
[[[1.07231727e-07 1.87638253e-01 5.02054952e-02 ... 9.97568748e-08
   1.07032086e-07 1.20764909e-07]
  [5.73666625e-07 8.70829001e-02 5.92710041e-02 ... 5.62546234e-07
   5.34377591e

In [55]:
import numpy as np

def convertPredictionToSentence(ds, tokenizer):
    finalSentence = np.zeros((len(ds), len(ds[0])), dtype='int32')
    for i, sentence in enumerate(ds):
        for j, word in enumerate(sentence):
            maxIndex = np.argmax(word, axis=0)
            print(maxIndex)
            finalSentence[i,j] = maxIndex

    return tokenizer.sequences_to_texts(finalSentence)


In [59]:
a = np.array([[[1,6,3,8,23,5,9,4,100],[1,6,3,8,3,5,9,4,50],[1,50,3,8,23,5,9,4,25]]])
convertPredictionToSentence(newSentence, simpDefTokenizer)


1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


['a a or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or or']

Predicting/Plotting Data

In [1]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()


NameError: name 'history' is not defined